## 1. Import Prerequisites

Run the following cell to confirm that the GPU is detected.

In [2]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')


Found GPU at: /device:GPU:0


In order for torch to use the GPU, we need to identify and specify the GPU as the device.   
Later, in our training loop, we will load data onto the device. 


In [3]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [4]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 36.1 MB/s 
     |████████████████████████████████| 636 kB 43.1 MB/s 
     |████████████████████████████████| 895 kB 44.3 MB/s 
     |████████████████████████████████| 3.3 MB 59.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## 2. Parse


### 2.1 Load from file

In [173]:
# Check that dataset file is available
import glob

version_files = glob.glob('./versions_*.csv')

if len(version_files) == 0:
  raise SystemError("Cant find any versions!!")


# TODO: get latest uploaded version..
version_file = version_files[0]
print(f"Using - {version_file}")


Using - ./versions_2.csv


In [174]:
import pandas as pd
pd.options.display.max_colwidth = 10000


# Load the dataset into a pandas dataframe.
df = pd.read_csv(version_file)


# Report the number of sentences.
print(f'Number of sentences: {df.shape[0]}')

# Report the number of articles.
print(f'Number of unique articles: {df.article_id.unique().shape[0]}')
print(f'Number of articles with more than 2 versions: {df.loc[df.version > 1].article_id.unique().shape[0]}')
print(f'Number of articles with too many versions: {df.loc[df.version > 10].article_id.unique().shape[0]}')


Number of sentences: 187531
Number of unique articles: 28097
Number of articles with more than 2 versions: 11555
Number of articles with too many versions: 148


In [175]:
# Drop dups - 

print(f'Amount of lines before: {df.shape[0]}')
df.drop_duplicates(subset=["title"], keep='first', inplace=True)
print(f'Amount of lines after: {df.shape[0]}')

Amount of lines before: 187531
Amount of lines after: 39273


### 2.1 Problem finding in data

In [119]:
df.article_id.value_counts()

1.9901095                                  37
1.9792445                                  31
1.9825326                                  27
1.9802921                                  24
1.9796542                                  21
                                           ..
https://fun.walla.co.il/item/3428734        1
864973 at https://www.israelhayom.co.il     1
1.9563029                                   1
1.9785704                                   1
1.9602937                                   1
Name: article_id, Length: 27848, dtype: int64

In [120]:
for problematic_article in df.loc[df.version > 15].article_id.unique():
  print(f"============== start: {problematic_article} =================")

  test_case = df.loc[df.article_id == "1.9792445"]

  test_case.sort_values(by=["version"], inplace=True)
  # test_case[["title", "version", "article_id"]]

  for _, r in test_case.iterrows():
    print(r.title)
  
  print(f"-------------- end {problematic_article} -----------------")

============== start: 1.9514062 =================
מטחים כבדים על אשקלון ועוטף עזה; היערכות לסבב של כמה ימי לחימה
מטחים כבדים לאשקלון; שני מבנים בעיר נפגעו ושלושה אנשים נפצעו
בתים באשקלון ספגו פגיעות ישירות; שבעה נפצעו, שניים בינוני
בתים באשקלון ספגו פגיעות ישירות; שבעה נפצעו, שניים במצב בינוני
בתים באשקלון ספגו פגיעות ישירות; שבעה נפצעו, בהם ארבעה בני משפחה אחת
אושר גיוס מילואים; דיווח בעזה: פעילים בג'יהאד נהרגו בתקיפת צה"ל
כוכבי: נערכים למבצע ללא מגבלת זמן; דיווח בעזה: פעילי ג'יהאד נהרגו בתקיפת צה"ל
כוכבי: נערכים למבצע ללא מגבלת זמן; דיווח בעזה: בכירי ג'יהאד נהרגו בתקיפת צה"ל
כוכבי: נערכים למבצע ללא מגבלת זמן; חמאס: הירי בהמשך יהיה עוצמתי יותר
רקטות פגעו באשדוד ובאשקלון; חמאס: הירי בהמשך יהיה עוצמתי יותר
פגיעות ישירות במבנים באשדוד ובאשקלון; חמאס: הירי בהמשך יהיה עוצמתי יותר
דובר צה"ל: תושבי אשקלון מתבקשים להישאר במרחב המוגן עד להודעה חדשה
שני פצועים בפגיעה ישירה במבנה באשקלון, אחת מהם במצב קשה
שתי הרוגות בפגיעות ישירות במבנים באשקלון, פצוע נוסף במצב קשה
שתי הרוגות בפגיעות ישירות במבנ

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


פגיעות ישירות במבנים באשדוד ובאשקלון; חמאס: הירי בהמשך יהיה עוצמתי יותר
דובר צה"ל: תושבי אשקלון מתבקשים להישאר במרחב המוגן עד להודעה חדשה
שני פצועים בפגיעה ישירה במבנה באשקלון, אחת מהם במצב קשה
שתי הרוגות בפגיעות ישירות במבנים באשקלון, פצוע נוסף במצב קשה
שתי הרוגות בפגיעות ישירות במבנים באשקלון, פצועה נוספת במצב קשה
אזעקות הופעלו בערי גוש דן והשרון
מטחים לעבר ערי המרכז: שלושה נפצעו באורח בינוני בחולון, בהם ילדה בת חמש
מטחים לעבר ערי המרכז: פצוע אנוש ושלושה פצועים בינוני בחולון
ישראל תחת מתקפה נרחבת: פצוע אנוש ושלושה פצועים בינוני בחולון
ישראל תחת מתקפה נרחבת: אישה נהרגה מפגיעת רקטה בראשון לציון
התפרעויות נרחבות בלוד: בן 56 נפצע קשה, עשרות מכוניות הוצתו
התפרעויות בלוד: יהודים הותקפו, עשרות מכוניות הוצתו; מג"ב ייכנס לעיר
התפרעויות בלוד: יהודים הותקפו, עשרות מכוניות הוצתו; מג"ב נכנס לעיר
התחדש הירי לעבר גוש דן והדרום; מצב חירום הוכרז בלוד
התחדש הירי לעבר המרכז, השפלה והדרום; מצב חירום בלוד בעקבות ההתפרעויות
התחדש הירי לעבר גוש דן והדרום; מצב חירום בלוד בעקבות ההתפרעויות
מטחים כבדים מהדרום

In [121]:


# Display 10 random rows from the data.
# df.loc[(df.article_id == "1.9812346")]
df.sort_values(by=["article_id"], inplace=True)

# df[["title", "version", "article_id"]].head(20)
# df.loc[df.version > 100][["title", "version", "article_id"]].article_id


df[["title", "version", "article_id"]].head(10)


# df.sample(10)


,title,version,article_id
167333,איך להכין סטייק בדקה ולברוח מהבית,1,1.1005838
37201,"סלט ירוק עם משמשים, גבינת עזים ואגוזים",1,1.1022818
125491,להקפיא את הזמן: אוכל לקיץ של אייל שני,1,1.1037029
180545,גספצ'ו אפרסקים ללא בישול,1,1.1045416
7881,"בקלנועית, בדרך לקרוסטיני",1,1.1048736
8043,מתכונים לקרוסטיני,2,1.1048736
5882,קומקוואט: תפוז בזעיר אנפין,1,1.1067876
177007,ממרק מיסו ועד מוקפץ: מה עושים עם אצות?,1,1.1087970
40473,מסע הצלב של אורי-בורי,1,1.1096986
20842,למצות את הפוטנציאל: שלושה מתכונים מפתיעים עם מצות,1,1.1098187


### 2.3 How many lines have a version ?

In [145]:
v = df.article_id.value_counts()
random_article = v[v == 11].sample()


random_article_id = list(random_article.to_dict().keys())[0]

df[df.article_id == random_article_id].sort_values(inplace=False, by=["version"])[["title", "version", "article_id"]]


,title,version,article_id
22819,ב-07:00 בבוקר: הקלפיות ייפתחו וישראל תצא להצביע בפעם הרביעית בתוך שנתיים,1,863781 at https://www.israelhayom.co.il
22822,הקלפיות ברחבי הארץ נפתחו: ישראל הולכת לבחירות רביעיות בתוך שנתיים,3,863781 at https://www.israelhayom.co.il
22833,ישראל הולכת לבחירות רביעיות בתוך שנתיים: קלפי למבודדים קרסה בבאר שבע,7,863781 at https://www.israelhayom.co.il
22890,"בחירות 2021 יצאו לדרך: קלפי למבודדים קרסה בבאר שבע, החלה ההצעה בנתב""ג",9,863781 at https://www.israelhayom.co.il
22893,"בחירות 2021 יצאו לדרך: קלפי למבודדים קרסה בבאר שבע, החלה ההצבעה בנתב""ג",10,863781 at https://www.israelhayom.co.il
22898,"בחירות 2021 יצאו לדרך: תלונה על תקיפת משקיפה בכפר ברע, קלפי למבודדים קרסה ב""ש",11,863781 at https://www.israelhayom.co.il
22900,"בחירות 2021 יצאו לדרך: תלונה על תקיפת משקיפה בצפון, קלפי למבודדים קרסה ב""ש",12,863781 at https://www.israelhayom.co.il
22901,"בחירות 2021 יצאו לדרך: תלונה על תקיפת משקיפה במרכז, קלפי למבודדים קרסה ב""ש",13,863781 at https://www.israelhayom.co.il
22916,בחירות 2021 יצאו לדרך: דקה אחר דקה,14,863781 at https://www.israelhayom.co.il
22937,"תלונות על פגיעה בטוהר הבחירות, אחוז ההצבעה נכון ל-10:00 - גבוה מהסבב הקודם",16,863781 at https://www.israelhayom.co.il


### Drop single version articles

In [176]:
single_version_article_ids = list(v[v == 1].to_dict().keys())

lines_to_be_removed = df[df.article_id.isin(single_version_article_ids)][["version", "title", "article_id"]]

print(f"Removing {lines_to_be_removed.size} lines!")

df.drop(df[df.article_id.isin(single_version_article_ids)].index, inplace=True)

print(f'Number of unique articles: {df.article_id.unique().shape[0]}')

Removing 59784 lines!
Number of unique articles: 7920


### 2.3 Almost done paring data

In [9]:
MAX_WORDS = df.amount_of_words.max()
print(f"Max words in sentence is - {MAX_WORDS}")


Max words in sentence is - 24.0


Let's extract the sentences and labels of our training set as numpy ndarrays.

In [10]:
sentences = df.title.values
labels = df.label.values

# 3. Tokenization & Input Formatting


## 3.1. BERT Tokenizer

In [27]:
from transformers import AutoTokenizer, AutoModelForMaskedLM


# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = AutoTokenizer.from_pretrained("avichr/heBERT")
# tokenizer = AutoTokenizer.from_pretrained("onlplab/alephbert-base")


Loading BERT tokenizer...


Let's apply the tokenizer to one sentence just to see the output.

In [28]:
import random

example_sentence = sentences[random.choice(range(len(sentences)))]
# Print the original sentence.
print(' Original: ', example_sentence)

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(example_sentence))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(example_sentence)))


 Original:  הציפייה לברברים היא הברבריות עצמה
Tokenized:  ['הצי', '##פייה', 'לבר', '##ברים', 'היא', 'הבר', '##בריו', '##ת', 'עצמה']
Token IDs:  [6033, 4281, 3444, 1843, 1667, 3464, 6576, 1019, 3523]


In [29]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'

                        # This function also supports truncation and conversion
                        # to pytorch tensors, but we need to do padding, so we
                        # can't use these features :( .
                        #max_length = 128,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

TypeError: ignored

In [ ]:
# We'll borrow the `pad_sequences` utility function to do this.
from keras.preprocessing.sequence import pad_sequences

# Set the maximum sequence length.
MAX_LEN = 32

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# Pad our input tokens with value 0.
# "post" indicates that we want to pad and truncate at the end of the sequence,
# as opposed to the beginning.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

print('\nDone.')


Padding/truncating all sentences to 32 values...

Padding token: "[PAD]", ID: 0

Done.


In [ ]:
# Create attention masks
attention_masks = []

# For each sentence...
for sent in input_ids:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

In [ ]:
# Use train_test_split to split our data into train and validation sets for
# training
from sklearn.model_selection import train_test_split

# Use 90% for training and 10% for validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)
# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2018, test_size=0.1)

In [ ]:
# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.

batch_size = 32

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


##????

In [ ]:
model = AutoModelForMaskedLM.from_pretrained("avichr/heBERT")

from transformers import pipeline
fill_mask = pipeline(
    "fill-mask",
    model="avichr/heBERT",
    tokenizer="avichr/heBERT"
)

options = fill_mask("הקורונה לקחה את [MASK] ולנו לא נשאר דבר.")

options

[{'score': 0.14047907292842865,
  'sequence': 'הקורונה לקחה את הילדים ולנו לא נשאר דבר.',
  'token': 3096,
  'token_str': 'הילדים'},
 {'score': 0.04530879110097885,
  'sequence': 'הקורונה לקחה את הכסף ולנו לא נשאר דבר.',
  'token': 5289,
  'token_str': 'הכסף'},
 {'score': 0.0362359881401062,
  'sequence': 'הקורונה לקחה את הכלב ולנו לא נשאר דבר.',
  'token': 12737,
  'token_str': 'הכלב'},
 {'score': 0.035021472722291946,
  'sequence': 'הקורונה לקחה את הילדה ולנו לא נשאר דבר.',
  'token': 12178,
  'token_str': 'הילדה'},
 {'score': 0.02997061051428318,
  'sequence': 'הקורונה לקחה את הרכב ולנו לא נשאר דבר.',
  'token': 3806,
  'token_str': 'הרכב'}]

In [ ]:
max(options, key=lambda x: x["score"])

{'score': 0.14047907292842865,
 'sequence': 'הקורונה לקחה את הילדים ולנו לא נשאר דבר.',
 'token': 3096,
 'token_str': 'הילדים'}